In [1]:
import os
from alphafold3_pytorch.utils.utils import default, exists, first
from alphafold3_pytorch import collate_inputs_to_batched_atom_input
from alphafold3_pytorch.alphafold3 import Alphafold3
from alphafold3_pytorch.inputs import (
    PDBDataset,
    molecule_to_atom_input,
    pdb_input_to_molecule_input,
)
from alphafold3_pytorch.data.weighted_pdb_sampler import WeightedPDBSampler


[01:26:37] UFFTYPER: Warning: hybridization set to SP3 for atom 0
[01:26:37] UFFTYPER: Unrecognized charge state for atom: 0
[01:26:37] UFFTYPER: Unrecognized atom type: Mn2+2 (0)
[01:26:37] UFFTYPER: Unrecognized atom type: Fe2+2 (0)
[01:26:37] UFFTYPER: Unrecognized atom type: Co3+3 (0)
[01:26:37] UFFTYPER: Unrecognized atom type: Ni3+2 (0)
[01:26:37] UFFTYPER: Unrecognized charge state for atom: 0
[01:26:37] UFFTYPER: Unrecognized atom type: Zn+2 (0)
[01:26:37] UFFTYPER: Unrecognized atom type: Ca+2 (0)


Loading CCD component SMILES strings from data/ccd_data/components_smiles.json.


/cpfs01/projects-HDD/cfff-6f3a36a0cd1e_HDD/public/protein/workspace/chengkaihui/conda_envs/af3/lib/python3.10/site-packages/rotary_embedding_torch/rotary_embedding_torch.py:35: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/cpfs01/projects-HDD/cfff-6f3a36a0cd1e_HDD/public/protein/workspace/chengkaihui/conda_envs/af3/lib/python3.10/site-packages/rotary_embedding_torch/rotary_embedding_torch.py:268: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/cpfs01/projects-HDD/cfff-6f3a36a0cd1e_HDD/public/protein/workspace/chengkaihui/conda_envs/af3/lib/python3.10/site-packages/local_attention/rotary.py:33: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/cpfs01/projects-HDD/cfff-6f3a36a0c

In [2]:
# data_test = os.path.join("data", "test")
data_test = os.path.join('/cpfs01/projects-HDD/cfff-6f3a36a0cd1e_HDD/public/protein/datasets/AF3/data/pdb_data/data_caches/200_mmcif')

"""Test a PDBDataset constructed using a WeightedPDBSampler."""
interface_mapping_path = os.path.join(data_test, "interface_cluster_mapping.csv")
chain_mapping_paths = [
    os.path.join(data_test, "ligand_chain_cluster_mapping.csv"),
    os.path.join(data_test, "nucleic_acid_chain_cluster_mapping.csv"),
    os.path.join(data_test, "peptide_chain_cluster_mapping.csv"),
    os.path.join(data_test, "protein_chain_cluster_mapping.csv"),
]

sampler = WeightedPDBSampler(
    chain_mapping_paths=chain_mapping_paths,
    interface_mapping_path=interface_mapping_path,
    batch_size=1,
)

dataset = PDBDataset(
    folder=os.path.join("/cpfs01/projects-HDD/cfff-6f3a36a0cd1e_HDD/public/protein/datasets/AF3/data/pdb_data", "merged_mmcifs"), 
    sampler=sampler, sample_type="default", crop_size=128
)
len(dataset)

2024-09-04 01:26:41.076 | INFO     | alphafold3_pytorch.data.weighted_pdb_sampler:__init__:225 - Precomputing chain and interface weights. This may take several minutes to complete.
2024-09-04 01:26:41.078 | INFO     | alphafold3_pytorch.data.weighted_pdb_sampler:__init__:258 - Finished precomputing chain and interface weights.


208

In [3]:
atom_num_cut_off = 2000
wanted_file_id = []
error_cnt=0
for i in range(len(dataset)):
    data = dataset[i]
    filepath = data.mmcif_filepath
    file_id = os.path.splitext(os.path.basename(filepath))[0] if exists(filepath) else None
    try:
        mol_input = pdb_input_to_molecule_input(pdb_input=data)
        atom_input = molecule_to_atom_input(mol_input)
        print(f"pass data:{i} | {file_id}",len(mol_input.molecules),len(atom_input.atom_inputs))
        if len(atom_input.atom_inputs)<atom_num_cut_off:
            wanted_file_id.append(file_id)
    except Exception as e:
        print(f"Error in {i}:{file_id}")
        print(f'Exception: {e}')
print(wanted_file_id)

pass data:0 | 6mwz-assembly1 169 1360
pass data:1 | 6nit-assembly1 39 869
pass data:2 | 2jod-assembly1 139 1116
pass data:3 | 2mtz-assembly1 205 1600
pass data:4 | 2mtz-assembly1 205 1600
pass data:5 | 1nk3-assembly1 95 1249
pass data:6 | 209d-assembly1 27 430
pass data:7 | 1nk9-assembly1 609 5215
pass data:8 | 6mw0-assembly1 5 40
pass data:9 | 5mt8-assembly1 192 1511
pass data:10 | 408d-assembly1 22 524
pass data:11 | 6mu4-assembly1 589 4872
pass data:12 | 216d-assembly1 20 396
pass data:13 | 3mrg-assembly1 384 3155
pass data:14 | 4jol-assembly1 142 1208
pass data:15 | 1njm-assembly1 3139 65371
pass data:16 | 216d-assembly1 20 396
pass data:17 | 207d-assembly1 46 728
pass data:18 | 108d-assembly1 17 401
pass data:19 | 3j0o-assembly1 1430 20184
pass data:20 | 215d-assembly1 14 350
pass data:21 | 4ni7-assembly1 33 639
pass data:22 | 207d-assembly1 46 728
pass data:23 | 3mxc-assembly1 105 868
pass data:24 | 421p-assembly1 336 2724
pass data:25 | 221l-assembly1 165 1299
pass data:26 | 115

: 

In [ ]:
mol_input = pdb_input_to_molecule_input(pdb_input=dataset[0])

In [ ]:
atom_input = molecule_to_atom_input(mol_input)
batched_atom_input = collate_inputs_to_batched_atom_input([atom_input], atoms_per_window=27)

In [ ]:
for key,value in batched_atom_input.dict().items():
    try:
        print(key,value.shape)
    except:
        print(key,value)
print(batched_atom_input.dict()['filepath'])

atom_inputs torch.Size([1, 25240, 3])
molecule_ids torch.Size([1, 3176])
molecule_atom_lens torch.Size([1, 3176])
atompair_inputs torch.Size([1, 935, 27, 54, 5])
additional_molecule_feats torch.Size([1, 3176, 5])
is_molecule_types torch.Size([1, 3176, 5])
is_molecule_mod torch.Size([1, 3176, 4])
additional_msa_feats torch.Size([1, 1, 3176, 2])
additional_token_feats torch.Size([1, 3176, 33])
templates None
msa None
token_bonds torch.Size([1, 3176, 3176])
atom_ids None
atom_parent_ids torch.Size([1, 25240])
atompair_ids None
template_mask None
msa_mask None
atom_pos torch.Size([1, 25240, 3])
missing_atom_mask torch.Size([1, 25240])
molecule_atom_indices torch.Size([1, 3176])
distogram_atom_indices torch.Size([1, 3176])
atom_indices_for_frame torch.Size([1, 3176, 3])
distance_labels None
resolved_labels torch.Size([1, 25240])
resolution torch.Size([1])
chains torch.Size([1, 2])
filepath ('/cpfs01/projects-HDD/cfff-6f3a36a0cd1e_HDD/public/protein/datasets/AF3/data/pdb_data/merged_mmcifs/j

In [ ]:
alphafold3 = Alphafold3(
        dim_atom_inputs=3,
        dim_atompair_inputs=5,
        atoms_per_window=27,
        dim_template_feats=44,
        num_dist_bins=38,
        confidence_head_kwargs=dict(pairformer_depth=1),
        template_embedder_kwargs=dict(pairformer_stack_depth=1),
        msa_module_kwargs=dict(depth=1),
        pairformer_stack=dict(depth=2),
        diffusion_module_kwargs=dict(
            atom_encoder_depth=1,
            token_transformer_depth=1,
            atom_decoder_depth=1,
        ),
    ).cuda()
input_data = {k: v.cuda() if v is not None else v for k, v in batched_atom_input.model_forward_dict().items()}

loss = alphafold3(**input_data)
loss.backward()

OutOfMemoryError: CUDA out of memory. Tried to allocate 4.81 GiB. GPU 0 has a total capacity of 79.35 GiB of which 3.47 GiB is free. Process 7256 has 75.88 GiB memory in use. Of the allocated memory 70.26 GiB is allocated by PyTorch, and 5.13 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

tensor(8.9960, device='cuda:0', grad_fn=<AddBackward0>)

In [ ]:
# from pathlib import Path
# folder= os.path.join("/cpfs01/projects-HDD/cfff-6f3a36a0cd1e_HDD/public/protein/datasets/AF3/data/pdb_data", "test_mmcifs")
# if isinstance(folder, str):
#     folder = Path(folder)
# sampler_pdb_ids = set(sampler.mappings.get_column("pdb_id").to_list())
# files = {
#     os.path.splitext(os.path.basename(filepath.name))[0]: filepath
#     for filepath in folder.glob(os.path.join("**", "*.cif"))
#     if os.path.splitext(os.path.basename(filepath.name))[0] in sampler_pdb_ids
# }
# files

{}

173540


: 